In [1]:
import process as p
import pandas as pd
import dmc

In [2]:
df = p.processed_data()
df = p.shuffle(df)

In [3]:
start, end, split = pd.Timestamp('2014-1-1'), pd.Timestamp('2014-12-31'), pd.Timestamp('2014-10-1')
df.orderDate = pd.to_datetime(df.orderDate)
mask = (df.orderDate >= start) & (df.orderDate <= end)
df_full = df[mask]
te_size = 5000
tr_size = 5000
X, Y, fts = dmc.transformation.transform_preserving_header(df_full, scaler=dmc.transformation.scale_features, binary_target=True)
train = X[:tr_size], Y[:tr_size]
test = X[tr_size:tr_size+te_size], Y[tr_size:tr_size+te_size]

/home/willi/Documents/run-dmc/venv/lib/python3.4/site-packages/pandas/core/generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
res_tree = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1], fts, dmc.classifiers.DecisionTree)

KeyboardInterrupt: 

In [ ]:
'''Calculate impact of features for n iterations'''
impact_map = dict()
for feature in fts:
        impact_map[feature] = 0.
for i in range(3):
    print(i)
    res_tree = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1], fts, dmc.classifiers.DecisionTree)
    for feature in fts:
            impact_map[feature] = impact_map[feature] + res_tree.decrement[feature]
import operator
sorted_impact_list = sorted(impact_map.items(), key=operator.itemgetter(1))
print(sorted_impact_list)


0
1


In [ ]:
res_tree[(res_tree.decrement < 0)]

In [ ]:
res_tree = dmc.evaluation.evaluate_without_one_feature(train[0], train[1], test[0], test[1], fts, sorted_impact_list[0][0], dmc.classifiers.DecisionTree)
print(res_tree)